# T2.4 Extraer con selenium datos de licitacións do Concello da Coruña

Queremos facer un código que garde nunha BBDD a Microsoft SQL Server a información dos expedientes de licitación do Concello da Coruña e nun directorio (no actual) as capturas de pantalla de cada expediente.

Para montar o Microsoft SQL Server sigue a guía de: https://jfsanchez.es/docs/docker-sqlserver-microsoft/

Debes empregar selenium e a páxina: https://contrataciondelestado.es

Debes navegar por ela coma se indica no documento.

Por cada elemento (expediente), meterse nel e facer captura de pantalla (automáticamente, coa API de selenium).

**Entrega:**

    O código nun jupyter notebook.
    A BBDD exportada de Microsoft SQL Server.
    Un zip coas capturas de pantalla que fixo selenium.


In [30]:
from webdriver_manager.firefox import GeckoDriverManager
GeckoDriverManager().install()

'C:\\Users\\david\\.wdm\\drivers\\geckodriver\\win64\\v0.34.0\\geckodriver.exe'

In [31]:

import argparse
from enum import Enum
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
import time
import random

In [32]:
def navegacion():
    driver = webdriver.Firefox()
    driver.get('https://contrataciondelestado.es')
    wait = WebDriverWait(driver, 10)
    element = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/label")))
    elem = driver.find_element(By.XPATH, '//div[@class="card-item"][1]')
    elem.click()
    wait = WebDriverWait(driver, 10)
    element = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/label")))
    elem=driver.find_element(By.XPATH, '//span[text()="Licitaciones"]')
    elem.click()
    wait = WebDriverWait(driver, 10)
    element = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/label")))
    elem=driver.find_element(By.XPATH, '//span[text()="Búsqueda avanzada"]')
    elem.click()
    wait = WebDriverWait(driver, 10)
    element = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/label")))
    elem=driver.find_element(By.XPATH, '//span[text()="Seleccionar"]')
    elem.click()
    wait = WebDriverWait(driver, 10)
    element = wait.until(EC.presence_of_element_located((By.XPATH, "//*[@id='viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:botonAnadirMostrarPopUpArbolEO']")))
    elem = driver.find_element(By.XPATH, '//div[text()="ENTIDADES LOCALES"]/../../td[@class="multiline"]')
    elem.click()

    elem = elem.find_elements(By.XPATH, '//div[text()="Galicia"]/../../td[@class="multiline"]')
    elem[1].click()

    elem2 = elem[1].find_elements(By.XPATH, '//div[text()="A Coruña"]/../../td[@class="multiline"]')
    elem2[2].click()

    elem_ayunt = elem2[2].find_elements(By.XPATH, '//div[text()="Ayuntamientos"]/../../td[@class="multiline"]')
    elem_ayunt[3].click()

    elem_ayunt2 = elem_ayunt[3].find_elements(By.XPATH, '//div[text()="A Coruña"]')
    elem_ayunt2[1].click()

    elem= elem_ayunt2[1].find_element(By.XPATH, '//span/select/option[1]')
    elem.click()
    elem.click()
    elem=driver.find_element(By.XPATH, '//*[@id="viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:botonAnadirMostrarPopUpArbolEO"]')
    elem.click()
    elem=driver.find_element(By.XPATH, '//*[@id="viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:button1"]')
    elem.click()
    return driver

In [33]:
# driver=navegacion()




Agora con beautifull soup procederemos a recoller todos os datos 

In [34]:
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

lista_final = []
contador = 0

def encontrar_pagina(driver):
    global contador  # Declaración para indicar que contador es global
    
    response = requests.get(driver.current_url)
    lista = []
    lista2 = []
    
    if response.status_code == 200:
        html = driver.page_source

        soup = BeautifulSoup(html, 'html.parser')

        tabla = soup.find('table', attrs={'id': 'myTablaBusquedaCustom'})

        if tabla:
            filas = tabla.find_all(['tr'])
            for fila in filas:
                celdas = fila.find_all(['td'])
                for celda in celdas:  
                    if 'Página' not in celda.text:
                        lista.append(celda.text.strip())
                lista2.append(lista)
        else:
            print("No se encontró la tabla en la página web.")

    else:
        print("La solicitud HTTP no fue exitosa. Código de estado:", response.status_code)
        
    elem = driver.find_elements(By.XPATH, '//table//a[2]/img')
    
    for ele in elem:
        ele.click()
        WebDriverWait(driver, 10).until(lambda driver: len(driver.window_handles) == 2)
        driver.switch_to.window(driver.window_handles[1])
        wait = WebDriverWait(driver, 10)
        element = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/label")))
        contador += 1
        driver.save_screenshot(f'im/a{contador}.png')
        
        wait = WebDriverWait(driver, 10)
        element = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/label")))

        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        
    return lista2


In [35]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
contador=0
def navegar_paginas(driver):
    wait = WebDriverWait(driver, 10)  
    span_element = wait.until(EC.presence_of_element_located((By.ID, "viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:textfooterInfoTotalPaginaMAQ")))
    valor = span_element.text

    for _ in range(int(valor)-1):
        lista_final.append(encontrar_pagina(driver))

        elem = driver.find_element(By.XPATH, '//*[@title="Siguiente"]')
        elem.click()
        wait.until(EC.presence_of_element_located((By.XPATH, "//*[@id='viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:footerUltimo']")))

        
    lista_final.append(encontrar_pagina(driver))
    

In [36]:
driver=navegacion()
navegar_paginas(driver)

TimeoutException: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:191:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:509:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16


In [ ]:
lista_simple = sum(lista_final, [])
lista_final = sum(lista_simple, [])

In [ ]:
lista_final

['211/2023/882Servicios de organización de la carrera popular Media Maratón Coruña C21 en la ciudad de A Coruña en el año 2024, que se va a realizar en esta ciudad el día 25 de febrero de 2024. Paralelamente y aprovechando la infraestructura del evento se organizará una carrera popular de 5 Km de recorrido que servirá como entrenamiento para el circuito de carrera.',
 'ServiciosServicios de esparcimiento, culturales y deportivos',
 'Resuelta',
 '21.659,40',
 'Publicación PLACSP:Adjudicación:12/02/2024Formalización:15/02/2024',
 'Junta de Gobierno del Ayuntamiento de A Coruña',
 'A31/2022/115Servicio de Soporte y Mantenimiento especializado para la Plataforma Socyal V3',
 'ServiciosServicios de mantenimiento y reparación',
 'Resuelta',
 '27.192,00',
 'Publicación PLACSP:Adjudicación:10/01/2024Formalización:14/02/2024',
 'Junta de Gobierno del Ayuntamiento de A Coruña',
 '370/2024/26Concierto de una operación de préstamo, con previa apertura de crédito, destinada a la financiación de inv

In [ ]:
import pandas as pd
filas = []
nombres_columnas = [
    'Expediente',
    'Tipo de Contrato',
    'Estado',
    'Importe',
    'Fechas',
    'Órgano de Contratación'
]



In [ ]:
import pandas as pd


# Nombres de las columnas
nombres_columnas = [
    'Expediente',
    'Tipo de Contrato',
    'Estado',
    'Importe',
    'Fechas',
    'Órgano de Contratación'
]
longitud_sublista = len(nombres_columnas)
sublistas = [lista_final[i:i+longitud_sublista] for i in range(0, len(lista_final), longitud_sublista)]
df = pd.DataFrame(sublistas, columns=nombres_columnas)

df

,Expediente,Tipo de Contrato,Estado,Importe,Fechas,Órgano de Contratación
0,211/2023/882Servicios de organización de la ca...,"ServiciosServicios de esparcimiento, culturale...",Resuelta,"21.659,40",Publicación PLACSP:Adjudicación:12/02/2024Form...,Junta de Gobierno del Ayuntamiento de A Coruña
1,A31/2022/115Servicio de Soporte y Mantenimient...,ServiciosServicios de mantenimiento y reparación,Resuelta,"27.192,00",Publicación PLACSP:Adjudicación:10/01/2024Form...,Junta de Gobierno del Ayuntamiento de A Coruña
2,370/2024/26Concierto de una operación de prést...,Privado,Evaluación,"22.763.291,51",Present. Solicitud:16/02/2024Present. Oferta:1...,Junta de Gobierno del Ayuntamiento de A Coruña
3,236/2023/57Procedimiento abierto simplificado ...,ServiciosServicios de transporte por vía terre...,Resuelta,"58.050,00",Publicación PLACSP:Adjudicación:11/01/2024Form...,Junta de Gobierno del Ayuntamiento de A Coruña
4,851/2023/103Contratación mixta de suministro e...,SuministrosAdquisición,Adjudicada,"231.404,96",Publicación PLACSP:Adjudicación:29/01/2024,Junta de Gobierno del Ayuntamiento de A Coruña
...,...,...,...,...,...,...
19418,"370/2018/60Contrato de préstamo, con previa ap...",Privado,Adjudicada,"7.975.450,00",Publicación PLACSP:Adjudicación:15/06/2018,Junta de Gobierno del Ayuntamiento de A Coruña
19419,106/2018/117Suministro de Gas Natural para cal...,SuministrosAdquisición,Evaluación,"28.099,17",Present. Oferta:28/06/2018,Junta de Gobierno del Ayuntamiento de A Coruña
19420,236/2018/41Obra de sustitución de la cubierta ...,ObrasConstrucción,Resuelta,"58.162,82",Publicación PLACSP:Adjudicación:31/05/2018Form...,Junta de Gobierno del Ayuntamiento de A Coruña
19421,541/2018/916Refuerzo de contenedores en la zon...,ServiciosServicios de alcantarillado y elimina...,Evaluación,"16.043,52",Present. Oferta:24/05/2018,Junta de Gobierno del Ayuntamiento de A Coruña


Error al conectar a la base de datos: ('28000', '[28000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Login failed for user \'sa\'. (18456) (SQLDriverConnect); [28000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Cannot open database "concellos" requested by the login. The login failed. (4060); [28000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Login failed for user \'sa\'. (18456); [28000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Cannot open database "concellos" requested by the login. The login failed. (4060)')


In [2]:
import pyodbc

connectionString = 'DRIVER={ODBC Driver 18 for SQL Server};SERVER=10.133.29.80,41433;DATABASE=concellos;UID=sa;PWD=Abc12300'
conexion = pyodbc.connect(connectionString)

In [3]:
cursor = conexion.cursor()
cursor.executemany("""INSERT INTO Concello(Expediente,Contrato,Estado,Importe,Fechas,Organo_Contratacion)Values (?,?,?,?,?,?)""",df.values.tolist())
cursor.commit()